# LSTM

- ## Preliminaries

- ### Imports

In [12]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout,TimeDistributed
from keras.layers import LSTM,SimpleRNN
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import os, os.path
from os import listdir
from os.path import isfile, join

- ### Check CPU usage

In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [3]:
get_available_gpus()

[u'/gpu:0']

----------

# I. Toy examples

- ### Test

** 1. Load and convert data**

'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

In [4]:
#load file
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
VOCAB_SIZE = len(chars)
print('total chars:',VOCAB_SIZE)

corpus length: 600901
total chars: 59


**Warning:** The RNN takes in input numerical data hence the necessity to convert strings into numerical values.

In [5]:
#creating mapping between indexes and characters
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

We’re gonna use Keras to create and train our Network, so we must convert the data into this form: (number_of_sequences, length_of_sequence, number_of_features).
- nb of features = length of the char array
- length of sequence = batch size
- nb of sequence = len(data) divided by batch size.

**Warning : ** target sequence is setted by shifting the source/input sequence by one character with both having the same length.

In [6]:
%%time

SEQ_LENGTH=100
#Build three dimensional arrays
X = np.zeros((len(text)/SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE)) #input
y = np.zeros((len(text)/SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE)) #target

#Build sequences
for i in range(0, len(text)/SEQ_LENGTH):
    X_sequence = text[i*SEQ_LENGTH:(i+1)*SEQ_LENGTH]
    X_sequence_ix = [char_indices[value] for value in X_sequence]
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence

    y_sequence = text[i*SEQ_LENGTH+1:(i+1)*SEQ_LENGTH+1]
    y_sequence_ix = [char_indices[value] for value in y_sequence]
    target_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1.
    y[i] = target_sequence

CPU times: user 460 ms, sys: 24 ms, total: 484 ms
Wall time: 484 ms


** 2. Build the network**

In [7]:
HIDDEN_DIM= 500 #500
LAYER_NUM = 2


model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [8]:
def generate_text(model, length, vocab_size, ix_to_char):
    # starting with random character
    ix = [np.random.randint(vocab_size)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, vocab_size))
    for i in range(length):
        # appending the last predicted character to sequence
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)

In [9]:
# Generate some sample before training to know how bad it is!
generate_text(model, 100, VOCAB_SIZE, indices_char)

d ----hh???�����44�99j((fnnnnn))��������::::::����:::::��,,::::��,,:::���:::��:::��::�::��:::�,,:::�

'd ----hh???\xa6\xa6\xa6\xa6\xa644\xa699j((fnnnnn))\xc3\x86\x86\x86\x86\x86\x86\x86::::::\x86\x86\x86\x86:::::\x86\x86,,::::\x86\x86,,:::\x86\x86\x86:::\x86\x86:::\x86\x86::\x86::\x86\x86:::\x86,,:::\x86,'

**3. Train network**

In [ ]:
#batch size equals to seq length here
BATCH_SIZE=100
#len of desired output
GENERATE_LENGTH=100
DIR="weight_03/"

try:
    onlyfiles = [f for f in listdir(DIR) if isfile(join(DIR, f))]
    nb_files = len(onlyfiles)
    name_template = onlyfiles[0][0:15]
except Exception as e:
    print(e)
    if not os.path.exists(DIR):
        os.makedirs(DIR)
    nb_files=0

if nb_files>0:
    nb_iteration=nb_files*10
    model.load_weights(DIR+onlyfiles[0][0:21]+str(nb_iteration)+'.hdf5')
else:
    nb_iteration=0
    
print("Starting at iteration : %s" %nb_iteration)
while True:
    print('\n')
    print('-'*20)
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=2, nb_epoch=1)
    nb_iteration += 1
    generate_text(model, GENERATE_LENGTH,VOCAB_SIZE, indices_char)
    if nb_iteration % 10 == 0:
        print("\n\nIteration nb : %s" %nb_iteration)
        model.save_weights(DIR+'checkpoint_{}_epoch_{}.hdf5'.format(HIDDEN_DIM, nb_iteration))

**4. Generate text**

In [43]:
%%time
out = generate_text(model, 1000, VOCAB_SIZE, indices_char)

5] of cruelty in the last tempt ougnt to another will or to an all inclusive rule and
ritual, pretty much as the brahmin leaves absolutely and
impression: the individual is
almost automatically bound to rule and tradition
from the contempt of one's fellow men.
there is perhaps just the behispociation of good fear and skeptics must
be taken into the working of false developed in responsible for artist.
each otherwise is made a mistake before us (or:--every virtue worthy distrust in everything extent in descal question, when it wants to be thre
tendency of his sentimen, the most conceited beeenism of the individual
purpose and approction that it is addinate the sake of one indern into contrasion of this life, self-denial. counting nor hatred and the like of the
too-rult" and its assuredly they will be men of the masters of the older, the
strength of my streem from the multitude, every system of morals in this
light: it is "deceptive space. 
with herristes and pre-cated with many our spea